# First Part

objectives: 

1. Scrape wikipedia for toronto boroughs. 
2. Pass data into dataframe
3. Clean data and sort into friendly format

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests
import pandas as pd

In [3]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup=BeautifulSoup(source, 'lxml')
article=soup.find('table')
print(article.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

extract the 'useful stuff' from the html

In [5]:
MyTable=soup.find('table', {'class':'wikitable sortable'})
#MyTable

In [6]:
content=MyTable.findAll('tr')
#content

In [7]:
data=[]
for row in content:
    data.append([t.text.strip() for t in row.findAll('td')])

df=pd.DataFrame(data,columns=['PostalCode', 'Borough', 'Neighbourhood'])
df=df[~df['PostalCode'].isnull()]

print(df.head())

  PostalCode           Borough     Neighbourhood
1        M1A      Not assigned      Not assigned
2        M2A      Not assigned      Not assigned
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront


Multiple lines are not usable due to a Borough not being assigned. We will drop these, and group boroughs according to Postal Code. We will also remove duplicates listed in the records.

In [8]:
df.drop(df[df['Borough']=='Not assigned'].index,axis=0,inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [9]:
df1=df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights


In [10]:
df2=df1.groupby("PostalCode").agg(lambda x:','.join(x))

Despite having a borough, some don't have a neighborhood. In these cases the neighbourhood will be the borough.

In [11]:
df2.loc[df2['Neighbourhood']=='Not assigned', 'Neighbourhood']=df2.loc[df2['Neighbourhood']=='Not assigned', 'Borough']

In [12]:
df3=df2.reset_index()

In [13]:
df3['Borough']=df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
df3.shape

(103, 3)

# Second Part

In [15]:
!pip install geocoder

     |████████████████████████████████| 102kB 15.4MB/s ta 0:00:01


In [17]:
import geocoder



In [19]:
postal_code='M1G'
g=geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
print(g.lat, g.lng)

43.768369121000035 -79.21758999999997


In [20]:
posts=[]
for post in df3['PostalCode']:
    g=geocoder.arcgis('{}, Toronto, Ontario'.format(post))
    posts.append((post,g.lat,g.lng))


[('M1B', 43.811525000000074, -79.19551746399998),
 ('M1C', 43.78566500000005, -79.15872457299997),
 ('M1E', 43.76581500000003, -79.17519294699997),
 ('M1G', 43.768369121000035, -79.21758999999997),
 ('M1H', 43.76968799900004, -79.23943999999995),
 ('M1J', 43.74312500000008, -79.23174973599998),
 ('M1K', 43.72627568400003, -79.26362499999993),
 ('M1L', 43.71305350000006, -79.28505499999994),
 ('M1M', 43.724234575000025, -79.22792499999997),
 ('M1N', 43.69677000000007, -79.25996735299998),
 ('M1P', 43.759975000000054, -79.26897418299995),
 ('M1R', 43.750710464000065, -79.30055999999996),
 ('M1S', 43.79394000000008, -79.26797613999997),
 ('M1T', 43.78472500000004, -79.29906603299997),
 ('M1V', 43.81768500000004, -79.28018721399997),
 ('M1W', 43.80088292800008, -79.32073999999994),
 ('M1X', 43.83421500000003, -79.21670085099998),
 ('M2H', 43.80284500000005, -79.35620744999994),
 ('M2J', 43.780970000000025, -79.34781328099996),
 ('M2K', 43.781015000000025, -79.38052867199997),
 ('M2L', 43.7

In [29]:
postaldf=pd.DataFrame(posts)
postaldf.columns=['PostalCode','Latitude','Longitude']
postaldf.head()

,PostalCode,Latitude,Longitude
0,M1B,43.811525,-79.195517
1,M1C,43.785665,-79.158725
2,M1E,43.765815,-79.175193
3,M1G,43.768369,-79.217590
4,M1H,43.769688,-79.239440


In [31]:
mergedf=pd.merge(df3,postaldf[['PostalCode','Latitude','Longitude']], on='PostalCode', how='left')
mergedf.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


# Third Part

In [33]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

In [36]:
latitude=43.811525
longitude=-79.195517

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(mergedf['Latitude'], mergedf['Longitude'], mergedf['Borough'], mergedf['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now connecting to Foursquare to obtain venues in vicinity of coordinates determined for each neighbourhood.

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names=mergedf['Neighbourhood'],
                                   latitudes=mergedf['Latitude'],
                                   longitudes=mergedf['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [43]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar
1,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
2,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
3,"Guildwood,Morningside,West Hill",43.765815,-79.175193,Heron Park,43.769327,-79.177201,Park
4,Woburn,43.768369,-79.217590,Starbucks,43.770037,-79.221156,Coffee Shop


In [42]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,13,13,13,13,13,13
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",13,13,13,13,13,13
"Alderwood,Long Branch",4,4,4,4,4,4
Bayview Village,5,5,5,5,5,5
"Bedford Park,Lawrence Manor East",20,20,20,20,20,20
Berczy Park,62,62,62,62,62,62
"Birch Cliff,Cliffside West",6,6,6,6,6,6


In [45]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 260 unique categories.


Now inserting OneHot coding for the 260 unique categories of venue.

In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfall,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Waterfall,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.00,0.030000,0.00,0.010000,0.000000,0.030000,0.000000,0.0,...,0.000000,0.0,0.010000,0.000000,0.000000,0.0,0.010000,0.000000,0.00,0.000000
1,Agincourt,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.00,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
4,"Alderwood,Long Branch",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
5,Bayview Village,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
6,"Bedford Park,Lawrence Manor East",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
7,Berczy Park,0.000000,0.00,0.000000,0.00,0.016129,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.016129,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
8,"Birch Cliff,Cliffside West",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000
9,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000


In [52]:
import numpy as np

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

With the top 10 venues, as follows, we will be able to run the clustering algorithm

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,Japanese Restaurant,Bar,American Restaurant,Bakery,Burger Joint,Breakfast Spot
1,Agincourt,Shopping Mall,Supermarket,Grocery Store,Sushi Restaurant,Bakery,Discount Store,Badminton Court,Pool,Chinese Restaurant,Vietnamese Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Liquor Store,Sandwich Place,Japanese Restaurant,Fried Chicken Joint,Coffee Shop,Beer Store,Pizza Place,Park,Fast Food Restaurant
4,"Alderwood,Long Branch",Convenience Store,Sandwich Place,Gym,Pub,Falafel Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Farm


In [71]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 0, 3, 3, 1, 3, 3, 3, 3], dtype=int32)

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = mergedf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')



,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725,4.0,Bar,Yoga Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193,1.0,Park,Gym / Fitness Center,Construction & Landscaping,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
3,M1G,Scarborough,Woburn,43.768369,-79.217590,1.0,Coffee Shop,Business Service,Korean Restaurant,Park,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,3.0,Playground,Trail,Farm,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dumpling Restaurant


It is necessary to drop the few rows that returned no venues, and then to transform the data type of the cluster labels to int.

In [84]:
toronto_merged=toronto_merged.dropna()

In [76]:
convert_dict={'Cluster Labels':int}
toronto_merged=toronto_merged.astype(convert_dict)

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Below are the clusters to visualize how they are composed. Cluster 1 seems to be the most eclectic, while 3 seems to have a large variety of restaurants.

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0,Pharmacy,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market


In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Park,Gym / Fitness Center,Construction & Landscaping,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
3,Scarborough,1,Coffee Shop,Business Service,Korean Restaurant,Park,Fast Food Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
19,NorthYork,1,Construction & Landscaping,Golf Driving Range,Dog Run,Park,Trail,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant
23,NorthYork,1,Convenience Store,Park,Bank,Speakeasy,Dance Studio,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
25,NorthYork,1,Food & Drink Shop,Bus Stop,Park,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
26,NorthYork,1,Park,Burger Joint,Coffee Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
30,NorthYork,1,Park,Airport,Food Court,Coffee Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
34,NorthYork,1,Park,Food Stand,Grocery Store,Yoga Studio,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
40,EastYork,1,Bar,Farmers Market,Italian Restaurant,Park,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
41,EastToronto,1,Bus Line,Grocery Store,Park,Discount Store,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm


In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,NorthYork,2,Home Service,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
94,Etobicoke,2,Home Service,Filipino Restaurant,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
96,NorthYork,2,Home Service,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market


In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,3,Playground,Trail,Farm,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Dumpling Restaurant
5,Scarborough,3,Restaurant,Indian Restaurant,Train Station,Women's Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
6,Scarborough,3,Chinese Restaurant,Hobby Shop,Coffee Shop,Department Store,Farmers Market,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
7,Scarborough,3,Intersection,Bus Line,Bakery,Coffee Shop,Soccer Field,Bus Station,Metro Station,Yoga Studio,Event Space,Falafel Restaurant
8,Scarborough,3,Sandwich Place,Discount Store,Pharmacy,Bank,Coffee Shop,Bistro,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
9,Scarborough,3,Gym Pool,Gym,College Stadium,Park,General Entertainment,Skating Rink,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space
10,Scarborough,3,Japanese Restaurant,Bakery,Light Rail Station,Gift Shop,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
11,Scarborough,3,Intersection,Auto Garage,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Electronics Store
12,Scarborough,3,Shopping Mall,Supermarket,Grocery Store,Sushi Restaurant,Bakery,Discount Store,Badminton Court,Pool,Chinese Restaurant,Vietnamese Restaurant
13,Scarborough,3,Pharmacy,Pizza Place,Thai Restaurant,Hobby Shop,Bus Stop,Chinese Restaurant,Shopping Mall,Fried Chicken Joint,Coffee Shop,Yoga Studio


In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4,Bar,Yoga Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
